In [11]:
#works with the quant file directly from MZMine
feat<-read.csv("quant.csv") #add in your quantfile name

#adjust these numbers
mz_tol<-0.001
rt_tol<-0.2 #set to what you used in MZMine
oversplit_num<-4 #lower if you want more removed

In [12]:
#you dont have to change anything here
feat$row.m.z<-round(feat$row.m.z, 4) 
feat$row.retention.time<-round(feat$row.retention.time, 2) 
mz_table<-as.data.frame(  table(feat$row.m.z)  )
potential_oversplits<-mz_table[mz_table$Freq > oversplit_num,]
potential_mz_forremov<-as.vector(potential_oversplits$Var1)
#these lines ID the potential oversplit MZs and saves them into the "potential_mz_forremov" vector. 
#If an MZ repeats more than the "oversplit num" set above, it gets saved into the vector
CIs_to_drop<-numeric()
for (i in 1:length(potential_mz_forremov)) {
  mz<-subset(feat, feat$row.m.z<=as.numeric(potential_mz_forremov[i])+mz_tol & feat$row.m.z>=as.numeric(potential_mz_forremov[i])-mz_tol)
  rt_sorted<-mz[order(mz$row.retention.time),]
  for (j in 2:nrow(rt_sorted)){
    dif<-rt_sorted$row.retention.time[j]-rt_sorted$row.retention.time[j-1]
    if (dif < rt_tol){
      CIs_to_drop<-c(CIs_to_drop, rt_sorted$row.ID[j])}}}
CIs_to_drop<-unique(CIs_to_drop)
cleaned_data<- subset(feat, !(row.ID %in% CIs_to_drop))
cleaned_data <- cleaned_data[, names(cleaned_data) != "X"]

#explaination of the nested-for-loop:The outer loop subsets the dataset with just a "potential mz for oversplit" value +/- the mz tolerance
#the subsetted dataset is sorted from low to high by RT value
#the inner loop finds the difference between each RT value, if the difference between the two is less than the "rt_tol" set above, the feature CI is saved in the "CIs to drop" vector


In [13]:
#if it gives an error that says: "(dif < rt_tol) { : missing value where TRUE/FALSE needed". lower the oversplit_num. It means nothing has been flagged as oversplit

In [14]:
length(CIs_to_drop) #how many values are removed

[1] 1

In [15]:
#data output line. put whatever name you want in
write.csv(cleaned_data, "OSremoved_quant.csv", row.names = FALSE) #make sure to keep .csv at end